In [4]:
# make a shapefile, triangular grid.
from osgeo import ogr, osr
import numpy as np
import os
print("Done")

Done


In [ ]:
A cell size of 1.0° would create triangles roughly 110 km wide
A cell size of 0.5° would create triangles roughly 55 km wide
A cell size of 0.1° would create triangles roughly 11 km wide
A cell size of 0.05° would create triangles roughly 5.5 km wide
A cell size of 0.01° would create triangles roughly 1.1 km wide

In [6]:
def create_triangular_grid_for_bounds(input_shapefile, output_shapefile, cell_size=0.05):
    """
    Creates a triangular grid within the bounds of the input shapefile
    Using WGS 1984 coordinate system (degrees)
    """
    # Open the input shapefile
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataSource = driver.Open(input_shapefile, 0)
    
    if dataSource is None:
        print(f"Could not open {input_shapefile}")
        return
    
    layer = dataSource.GetLayer()
    
    # Get the extent of the shapefile
    extent = layer.GetExtent()
    xmin, xmax, ymin, ymax = extent[0], extent[1], extent[2], extent[3]
    print(f"Shapefile bounds: xmin={xmin}, xmax={xmax}, ymin={ymin}, ymax={ymax}")
    
    # Get the spatial reference of the input
    spatialRef = layer.GetSpatialRef()
    
    # Calculate height of equilateral triangle in degrees
    height = cell_size * np.sqrt(3) / 2
    
    # Create the output shapefile
    if os.path.exists(output_shapefile):
        driver.DeleteDataSource(output_shapefile)
    
    outDataSource = driver.CreateDataSource(output_shapefile)
    outLayer = outDataSource.CreateLayer("triangular_grid", spatialRef, ogr.wkbPolygon)
    
    # Add ID field
    idField = ogr.FieldDefn("ID", ogr.OFTInteger)
    outLayer.CreateField(idField)
    
    # Generate triangles
    cell_id = 0
    
    # Make the grid slightly larger to ensure coverage
    xmin -= cell_size
    ymin -= height
    xmax += cell_size
    ymax += height
    
    # Number of cells in each direction
    nx = int((xmax - xmin) / cell_size) + 1
    ny = int((ymax - ymin) / height) + 1
    
    print(f"Creating grid with {nx}x{ny} cells...")
    
    # Create a consistent grid of equilateral triangles
    for i in range(ny):
        if i % 10 == 0:
            print(f"Processing row {i}/{ny-1}...")
            
        # Offset every other row to create proper equilateral triangles
        row_offset = (i % 2) * (cell_size / 2)
        
        for j in range(nx):
            # Base coordinates (with offset for every other row)
            x = xmin + j * cell_size + row_offset
            y = ymin + i * height
            
            # Create upward-pointing triangle
            ring = ogr.Geometry(ogr.wkbLinearRing)
            ring.AddPoint(x, y)
            ring.AddPoint(x + cell_size, y)
            ring.AddPoint(x + cell_size/2, y + height)
            ring.CloseRings()
            
            poly = ogr.Geometry(ogr.wkbPolygon)
            poly.AddGeometry(ring)
            
            # Add to layer
            outFeature = ogr.Feature(outLayer.GetLayerDefn())
            outFeature.SetGeometry(poly)
            outFeature.SetField("ID", cell_id)
            outLayer.CreateFeature(outFeature)
            outFeature = None
            cell_id += 1
            
            # Create downward-pointing triangle (if not on the bottom row)
            if i > 0:
                ring = ogr.Geometry(ogr.wkbLinearRing)
                ring.AddPoint(x, y)
                ring.AddPoint(x + cell_size/2, y - height)
                ring.AddPoint(x - cell_size/2, y - height)
                ring.CloseRings()
                
                poly = ogr.Geometry(ogr.wkbPolygon)
                poly.AddGeometry(ring)
                
                # Add to layer
                outFeature = ogr.Feature(outLayer.GetLayerDefn())
                outFeature.SetGeometry(poly)
                outFeature.SetField("ID", cell_id)
                outLayer.CreateFeature(outFeature)
                outFeature = None
                cell_id += 1
    
    # Clean up
    dataSource = None
    outDataSource = None
    
    print(f"Created triangular grid with {cell_id} cells at {output_shapefile}")
    return output_shapefile

# Example usage
input_shapefile = "/nesi/nobackup/uoo03627/qt_rat_sequencing/feems/LakeWhakatipushapefiles/POLYGON.shp"
output_shapefile = "/nesi/nobackup/uoo03627/qt_rat_sequencing/feems/LakeWhakatipushapefiles/LW_triangular_grid_0.01.shp"

# For WGS84 (degrees), 0.5 degrees is about 55 km at NZ's latitude
create_triangular_grid_for_bounds(input_shapefile, output_shapefile, cell_size=0.01)

Shapefile bounds: xmin=167.8263475694613, xmax=169.4144164087366, ymin=-45.59320859501844, ymax=-44.49839984522502
Creating grid with 161x129 cells...
Processing row 0/128...
Processing row 10/128...
Processing row 20/128...
Processing row 30/128...
Processing row 40/128...
Processing row 50/128...
Processing row 60/128...
Processing row 70/128...
Processing row 80/128...
Processing row 90/128...
Processing row 100/128...
Processing row 110/128...
Processing row 120/128...
Created triangular grid with 41377 cells at /nesi/nobackup/uoo03627/qt_rat_sequencing/feems/LakeWhakatipushapefiles/LW_triangular_grid_0.01.shp


'/nesi/nobackup/uoo03627/qt_rat_sequencing/feems/LakeWhakatipushapefiles/LW_triangular_grid_0.01.shp'